In [21]:
import os
import itertools
import numpy as np
import time
import re
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from scipy.spatial import cKDTree
from shapely import Point, LineString, get_coordinates
import openpyxl
import rasterio
import re
from rasterio.warp import transform, calculate_default_transform

In [22]:
ideam_dir='/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/2023/wwf/Inputs'
output_dir='/Users/carolinacastroosorio/Library/CloudStorage/OneDrive-UniversidaddelosAndes/2023/wwf/Outputs'
vector_dir=r'C:\Users\ccastro\OneDrive - Universidad de los andes\2023\wwf\Outputs'

In [4]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 8.7 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [25]:
os.chdir(ideam_dir)

#Guardamos todos los archivos de Hansen que abarcan Colombia en una lista.
defo_files=[]
for i in os.listdir():
    if '.tif' not in i:
        continue
    else:
        data=rasterio.open(i, 'r')
        defo_files.append(data)

In [26]:
defo_files

[<open DatasetReader name='20002002.tif' mode='r'>,
 <open DatasetReader name='20022004.tif' mode='r'>]

In [42]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling


# Definir el CRS objetivo (CRS 3116)
target_crs = 'EPSG:3116'

os.chdir(ideam_dir)

# Guardamos todos los archivos de Hansen que abarcan Colombia en una lista.
defo_files = []
for i in os.listdir():
    if i.endswith('.tif'):
        data = rasterio.open(i, 'r')
        defo_files.append(data)

# Reproyectar y guardar los rasters en el nuevo CRS
for data in defo_files:
    transform, width, height = calculate_default_transform(data.crs, target_crs, data.width, data.height, *data.bounds)
    kwargs = data.meta.copy()
    kwargs.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    output_filename = os.path.join(output_dir, f"reprojected_{os.path.basename(data.name)}")
    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        for i in range(1, data.count + 1):
            reproject(
                source=rasterio.band(data, i),
                destination=rasterio.band(dst, i),
                src_transform=data.transform,
                src_crs=data.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )


In [19]:
os.chdir(output_dir)

#Guardamos todos los archivos de Hansen que abarcan Colombia en una lista.
defo_files=[]
for i in os.listdir():
    if '.tif' not in i:
        continue
    else:
        data=rasterio.open(i, 'r')
        defo_files.append(data)


In [20]:
defo_files

[]

In [27]:
start = time.time()

for f in defo_files:
    project=re.search('(.*?)(?=.tif)', f.name).group(0).replace('', '_')
    file_name='/Points_{}.parquet'.format(project)
    raster=f.read(1)

    # Obtener las coordenadas de los puntos con valor mayor a cero
    points = np.argwhere(raster==2)
    # Dejar la columna de dato que va de '1 a 100'
    values = raster[points[:, 0], points[:, 1]]

    # Convertir las coordenadas de píxeles a coordenadas espaciales
    transform = f.transform
    lon, lat = rasterio.transform.xy(transform, points[:, 1], points[:, 0])

    # Crear un GeoDataFrame de los puntos
    gdf = gpd.GeoDataFrame({"geometry":gpd.points_from_xy(lon, lat), "pixel_type":values})
    gdf.crs = f.crs
    gdf=gdf.to_crs(epsg=3116)
    gdf.to_parquet(vector_dir+file_name)

end = time.time()

FileNotFoundError: [Errno 2] Failed to open local file 'C:\Users\ccastro\OneDrive - Universidad de los andes\2023\wwf\Outputs/Points__2_0_0_0_2_0_0_2_.parquet'. Detail: [errno 2] No such file or directory

In [35]:
defo_files

[]

In [22]:
print(Points__2_0_0_0_2_0_0_2_.head())

NameError: name 'Points__2_0_0_0_' is not defined